In [259]:
import pandas as pd

In [260]:
df = pd.read_csv("../regex/data/raw/domain_del.csv", encoding='latin1')

In [261]:
df.columns

Index(['id', 'promoType', 'price', 'hasVideo', 'agentNames', 'brandName',
       'addressStreet', 'addressSuburb', 'addressState', 'addressPostcode',
       'addressLat', 'addressLng', 'num_bath', 'type', 'formatted', 'isRural',
       'landSize', 'Retirement'],
      dtype='object')

In [262]:
# 清理之前的dataset numbers
price_columns_before = df['id'].count()

# inspect the price columns
df['price']

0       $500 per week
1       $540 per Week
2             $650.00
3       $560 per week
4                $600
            ...      
4909    $470 per week
4910             $600
4911             $575
4912    $530 per week
4913             $530
Name: price, Length: 4914, dtype: object

## Step 1 : Extract the price by Regax

In [263]:
# 过滤掉只含有文字列 提取数字
df['price'] = df['price'].str.extract(r'(\d+)')

# transfer to float
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# 考虑到爬虫下来的信息,含有每周车位费和房子卖价, 最高车位费为$100/pw
df = df[(df['price'] >= 170) & (df['price'] <= 2400)]

In [264]:
df['price']

0       500.0
1       540.0
2       650.0
3       560.0
4       600.0
        ...  
4909    470.0
4910    600.0
4911    575.0
4912    530.0
4913    530.0
Name: price, Length: 4402, dtype: float64

In [265]:
df['price'].describe()

count    4402.000000
mean      609.269196
std       199.719671
min       170.000000
25%       500.000000
50%       575.000000
75%       690.000000
max      2400.000000
Name: price, dtype: float64

In [266]:
# whole dataset instances
print(f"Before cleaning on price columns, there are {price_columns_before} records in the main dataset.")
print(f"After cleaning on price columns, there are {df['id'].count()} records in the main dataset.")

Before cleaning on price columns, there are 4914 records in the main dataset.
After cleaning on price columns, there are 4402 records in the main dataset.


## Save the dataset

In [267]:
df.to_csv('../regex/data/curated/Price_clean_domain.csv', index=False)